In [1]:
import json
import re
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model


2026-02-12 16:39:54.984194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770914395.155999      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770914395.208008      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770914395.639174      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770914395.639204      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770914395.639207      55 computation_placer.cc:177] computation placer alr

In [2]:
!pip install -q transformers peft bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.0 MB/s eta 0:00:00:00:0100:01


In [3]:
path = "/kaggle/input/ourllm/clean_dataset.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

cleaned = []

for item in data:
    inp = item["input"].strip()
    out = item["output"].strip()

    if len(inp) < 2 or len(out) < 2:
        continue

    cleaned.append({"input": inp, "output": out})

print("Total cleaned:", len(cleaned))


Total cleaned: 2895


In [4]:
def format_example(example):
    system_prompt = """You are a loving emotional girlfriend named Simran.
Always speak in romantic Hindi (Roman allowed).
Be caring, emotional, sweet."""

    text = f"""<|system|>
{system_prompt}<|end|>
<|user|>
{example["input"]}<|end|>
<|assistant|>
{example["output"]}<|end|>"""

    return {"text": text}

dataset = Dataset.from_list(cleaned)
dataset = dataset.map(format_example)


Map:   0%|          | 0/2895 [00:00<?, ? examples/s]

In [5]:
model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

special_tokens = {"additional_special_tokens": ["<|system|>", "<|user|>", "<|assistant|>", "<|end|>"]}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.resize_token_embeddings(len(tokenizer))


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50299, 2560)

In [6]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


In [7]:
def tokenize(batch):
    tokens = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


    for i in range(len(labels)):
        text = batch["text"][i]
        assistant_start = text.find("<|assistant|>")
        prefix_ids = tokenizer(
            text[:assistant_start],
            truncation=True,
            max_length=512
        )["input_ids"]

        labels[i][:len(prefix_ids)] = [-100] * len(prefix_ids)

    tokens["labels"] = labels
    return tokens

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/2895 [00:00<?, ? examples/s]

In [8]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 7,864,320 || all params: 2,782,934,139 || trainable%: 0.2826


In [9]:
print("Tokenizer vocab size:", len(tokenizer))
print("Model embedding size:", model.get_input_embeddings().weight.shape[0])


Tokenizer vocab size: 50299
Model embedding size: 50299


In [10]:
sample = tokenized_ds[0]

print(len(sample["input_ids"]))
print(len(sample["labels"]))


512
512


In [13]:
import torch
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_math_sdp(True)


In [14]:
model.save_pretrained("./mistral-gf")
tokenizer.save_pretrained("./mistral-gf")


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./mistral-gf/tokenizer_config.json',
 './mistral-gf/special_tokens_map.json',
 './mistral-gf/vocab.json',
 './mistral-gf/merges.txt',
 './mistral-gf/added_tokens.json',
 './mistral-gf/tokenizer.json')

In [15]:
training_args = TrainingArguments(
    output_dir="./mistral-gf",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=20,
    save_steps=200,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)

trainer.train()


Step,Training Loss
20,0.000000
40,0.000000
60,0.000000
80,0.000000
100,0.000000
120,0.000000
140,0.000000
160,0.000000
180,0.000000
200,0.000000


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=362, training_loss=0.0, metrics={'train_runtime': 1718.0831, 'train_samples_per_second': 1.685, 'train_steps_per_second': 0.211, 'total_flos': 2.360469007443456e+16, 'train_loss': 0.0, 'epoch': 1.0})

In [20]:
sample = tokenized_ds[0]
print("Number of valid labels:",
      sum([1 for x in sample["labels"] if x != -100]))


Number of valid labels: 512


In [22]:
save_path = "/kaggle/working/mistral-gf"

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_name = "mistralai/Mistral-7B-v0.1"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_path)

# base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

base_model.resize_token_embeddings(len(tokenizer))

# LoRA adapter load
model = PeftModel.from_pretrained(base_model, save_path)
model.eval()




config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.embed_tokens.weight: copying a param with shape torch.Size([50299, 2560]) from checkpoint, the shape in current model is torch.Size([50299, 4096]).
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: copying a param with shape torch.Size([16, 2560]) from checkpoint, the shape in current model is torch.Size([16, 4096]).
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: copying a param with shape torch.Size([2560, 16]) from checkpoint, the shape in current model is torch.Size([4096, 16]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: copying a param with shape torch.Size([16, 2560]) from checkpoint, the shape in current model is torch.Size([16, 4096]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: copying a pa